# Imports

In [5]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
# Importando modulo KNN
from sklearn.neighbors import KNeighborsClassifier

# Carregar Dataset

In [6]:
# Criando dataframe
df_dataset = pd.read_json('sample_data/data.json')
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   playlists  1000 non-null   object
dtypes: object(1)
memory usage: 7.9+ KB


In [7]:
df_dataset.shape

(1000, 1)

In [8]:
df_dataset.head(5)

,playlists
0,"{'name': 'Throwbacks', 'collaborative': 'false..."
1,"{'name': 'Awesome Playlist', 'collaborative': ..."
2,"{'name': 'korean ', 'collaborative': 'false', ..."
3,"{'name': 'mat', 'collaborative': 'false', 'pid..."
4,"{'name': '90s', 'collaborative': 'false', 'pid..."


# Normalização de Dados

**Filtrando músicas das playlist**

In [9]:
with open('sample_data/data.json','r') as f:
    data = json.loads(f.read())

# Carregando somente as playlists
df_playlist = pd.json_normalize(data, record_path =['playlists'])

In [10]:
# Quantidade de playlists / Quantidade de atributos de cada playlist
df_playlist.shape

(1000, 12)

In [11]:
# Informações de cada playlist
df_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           1000 non-null   object
 1   collaborative  1000 non-null   object
 2   pid            1000 non-null   int64 
 3   modified_at    1000 non-null   int64 
 4   num_tracks     1000 non-null   int64 
 5   num_albums     1000 non-null   int64 
 6   num_followers  1000 non-null   int64 
 7   tracks         1000 non-null   object
 8   num_edits      1000 non-null   int64 
 9   duration_ms    1000 non-null   int64 
 10  num_artists    1000 non-null   int64 
 11  description    20 non-null     object
dtypes: int64(8), object(4)
memory usage: 93.9+ KB


In [12]:
# Listando as 5 primeiras playlists
df_playlist.head(5)

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37,NaN
1,Awesome Playlist,false,1,1506556800,39,23,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",5,11656470,21,NaN
2,korean,false,2,1505692800,64,51,1,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",18,14039958,31,NaN
3,mat,false,3,1501027200,126,107,1,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",4,28926058,86,NaN
4,90s,false,4,1401667200,17,16,2,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",7,4335282,16,NaN


**Agrupar músicas**

In [13]:
# Número de músicas de todas(0-999) as playlists
df_playlist['num_tracks'].sum()

67503

In [14]:
# Acessando nome das playlists
nomes_playlist = df_playlist['pid']
nomes_playlist[1]

1

In [ ]:
# Acessando nome das playlists
nomes_playlist = df_playlist['name']
nomes_playlist

In [16]:
df_playlist_name = pd.DataFrame()
df_playlist_name["playlist_name"] = nomes_playlist 
df_playlist_name

,playlist_name
0,Throwbacks
1,Awesome Playlist
2,korean
3,mat
4,90s
...,...
995,old
996,Daze
997,rap
998,Country


In [ ]:
# Acessando musicas das playlists
musicas_playlist = df_playlist['tracks']
musicas_playlist[0]

In [259]:
def addDF(id_playlist, titulo, musicas):
  df = pd.DataFrame()  
  df = pd.DataFrame(musicas); 
  df['idPlaylist'] = id_playlist
  df['tituloPlaylist'] = titulo
  return df

In [260]:
def agrupar_musicas(df_playlist, qtdePlaylist):
  df = pd.DataFrame() 
  for i in range(qtdePlaylist):
    id_playlist = df_playlist['pid'][i]
    titulo = df_playlist['name'][i]
    musicas = df_playlist['tracks'][i]
    df = df.append(addDF(id_playlist, titulo, musicas))
  return df

In [261]:
df_musicas = agrupar_musicas(df_playlist, 2)
df_musicas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91 entries, 0 to 38
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   pos             91 non-null     int64 
 1   artist_name     91 non-null     object
 2   track_uri       91 non-null     object
 3   artist_uri      91 non-null     object
 4   track_name      91 non-null     object
 5   album_uri       91 non-null     object
 6   duration_ms     91 non-null     int64 
 7   album_name      91 non-null     object
 8   idPlaylist      91 non-null     int64 
 9   tituloPlaylist  91 non-null     object
dtypes: int64(3), object(7)
memory usage: 7.8+ KB


In [262]:
df_musicas

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,idPlaylist,tituloPlaylist
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,0,Throwbacks
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,0,Throwbacks
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),0,Throwbacks
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,0,Throwbacks
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,0,Throwbacks
...,...,...,...,...,...,...,...,...,...,...
34,34,Rush,spotify:track:3lpDrxUkr0tIe1kmJvdK7d,spotify:artist:2Hkut4rAAyrQxRdof7FVJq,YYZ,spotify:album:2xg7iIKoSqaDNpDbJnyCjY,265840,Moving Pictures,1,Awesome Playlist
35,35,Rush,spotify:track:0K6yUnIKNsFtfIpTgGtcHm,spotify:artist:2Hkut4rAAyrQxRdof7FVJq,Limelight,spotify:album:2xg7iIKoSqaDNpDbJnyCjY,259839,Moving Pictures,1,Awesome Playlist
36,36,Rush,spotify:track:0hBby0yygBY1u3m6tSpZgC,spotify:artist:2Hkut4rAAyrQxRdof7FVJq,The Camera Eye,spotify:album:2xg7iIKoSqaDNpDbJnyCjY,658986,Moving Pictures,1,Awesome Playlist
37,37,Rush,spotify:track:6cbBAD1AAYVaQR1cx89AZX,spotify:artist:2Hkut4rAAyrQxRdof7FVJq,Witch Hunt,spotify:album:2xg7iIKoSqaDNpDbJnyCjY,285533,Moving Pictures,1,Awesome Playlist


In [ ]:
  #print(len(listaMusicas))

  #for i in range(qtdePlaylists):
  #cont = 0
  #for i in df_playlist.index:  
    #nome_playlist = df_playlist['name'][i]
  # for i in range(qtdePlaylists):
  #   playlistTitulo = df_playlist['name'][i]
  #   print(playlistTitulo)
  #   dfMusica = pd.DataFrame(df_playlist['tracks'][i]);     
  #   df = pd.concat([df, dfMusica], ignore_index = True, axis = 0)
  #   df.at[i,'TituloPlaylist'] = playlistTitulo
    #for j in range(len(dfMusica)): 
  #  df.at[i,'playlistTitle'] = nome_playlist 
      #df.at[i,'trackName'] = dfMusica.loc[j, "track_name"] 
      #print(dfMusica.loc[i,"track_name"])
    #if (cont < qtdePlaylists):
      #for row in playlists_musicas[i]:
        #print(row['artist_name'])
        # trackName = row[cont]['track_name']
        # df.at[cont,'artistName'] = row['artist_name']
        # df.at[cont,'trackName'] = trackName
        # cont = cont + 1
    #df = df.append(playlists_musicas[i], ignore_index = True)
    #df.at[0,'playlistName'] = nome_playlist
    #df['playlistName'] = nome_playlist
  # for i in range(qtdePlaylists):    
  #   nome_playlist = df_playlist['name'][i]
  #   print(nome_playlist)
  #   for row in playlists_musicas[i]:         
  #     df_musicas = df_musicas.append(pd.DataFrame(row, index=[0]),  ignore_index=True)
  #     df_musicas['playlist_name'] = nome_playlist
  #   print('next')

In [86]:
def agrupar_musicas(playlists_musicas, qtdePlaylists, df_playlist):

  df = pd.DataFrame() 
  
  for i in range(qtdePlaylists):    
    print(df_playlist['name'][i])
    df = df.append(playlists_musicas[i], ignore_index = True)
    df['playlistName'] = df_playlist['name'][i]

  return df

In [ ]:
#BKP
def agrupar_musicas(playlists_musicas, qtdePlaylists):
  df_musicas = pd.DataFrame()

  for i in range(qtdePlaylists):    
    
    for row in musicas_playlist[i]:                  
      df_musicas = df_musicas.append(pd.DataFrame(row, index=[0]),  ignore_index=True)

  return df_musicas

In [ ]:
df_musica_ordenado = df_musicas.sort_values(by='artist_name', ascending=True)
df_musica_ordenado.head(20)

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
742,0,311,spotify:track:0HDaKOlVAfUWXdFR2RhBtN,spotify:artist:41Q0HrwWBtuUkJc7C1Rp6K,Down,spotify:album:6VIeIM5tetuv7FrYxISffU,172160,311
380,2,311,spotify:track:0orNm6qQT2B5WFi0l7l3PV,spotify:artist:41Q0HrwWBtuUkJc7C1Rp6K,Jackolantern's Weather,spotify:album:6VIeIM5tetuv7FrYxISffU,204160,311
451,4,311,spotify:track:2PGaYCles9SXXCjYOcjyfk,spotify:artist:41Q0HrwWBtuUkJc7C1Rp6K,Too Much To Think,spotify:album:5UWEPNU1vVbtxCBvI0ptOW,234451,MOSAIC
563,49,A Boogie Wit da Hoodie,spotify:track:3AMnOVxEHUgVrythGgqPhI,spotify:artist:31W5EY0aAly4Qieq6OFu6I,Say A',spotify:album:3HHp5I6Q6SEyU5bkvoCtnV,214853,The Bigger Artist
535,21,A Boogie Wit da Hoodie,spotify:track:1f5cbQtDrykjarZVrShaDI,spotify:artist:31W5EY0aAly4Qieq6OFu6I,Drowning (feat. Kodak Black),spotify:album:3HHp5I6Q6SEyU5bkvoCtnV,209269,The Bigger Artist
663,77,A Tribe Called Quest,spotify:track:0eEXcw3JLVXcRxYrVYMy68,spotify:artist:09hVIj6vWgoCDtT03h8ZCa,Electric Relaxation,spotify:album:3FAK4h4t9uV4JxzESXllAU,226133,The Anthology
194,39,A.Dd+,spotify:track:3tvb5DvwShgHDcpumThLU4,spotify:artist:0oiw7PY6VPt7w2KsV9Xhcu,Can't Come Down,spotify:album:7HgnwHkCSXCbStpPSUoCaV,224549,Can't Come Down
911,59,AFI,spotify:track:3fttmSWGThBQTNkuHMoCTN,spotify:artist:19I4tYiChJoxEO5EuviXpz,Girl's Not Grey,spotify:album:1eIzVBHA5NvX0wo2nLACew,190493,Sing The Sorrow
584,70,AJR,spotify:track:2pwnEzgIzYL4AOw4ousjkB,spotify:artist:6s22t5Y3prQHyaHWUN1R1C,Let the Games Begin,spotify:album:1qrqSSrDtAaA4fINxD0vtE,201004,Let the Games Begin
582,68,AJR,spotify:track:34j0oTMqyoEPJdtnCZS5vD,spotify:artist:6s22t5Y3prQHyaHWUN1R1C,Drama,spotify:album:7LACXphpLTluKLFqHIZ1Qq,204426,The Click


**Pré-processamento**

In [ ]:
df_musica_ordenado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1189 entries, 742 to 860
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pos          1189 non-null   int64 
 1   artist_name  1189 non-null   object
 2   track_uri    1189 non-null   object
 3   artist_uri   1189 non-null   object
 4   track_name   1189 non-null   object
 5   album_uri    1189 non-null   object
 6   duration_ms  1189 non-null   int64 
 7   album_name   1189 non-null   object
dtypes: int64(2), object(6)
memory usage: 83.6+ KB


In [ ]:
from sklearn.preprocessing import LabelEncoder

#Transform fields from Object to int
def transform_fields(df, fields):
  label_encoder = LabelEncoder()
  for field in fields:
    df[field] = label_encoder.fit_transform(list(df[field].values))
  return df

In [ ]:
referencia_nome_artista = ['artist_uri']
df_musicas_formatada = transform_fields(df_musica_ordenado, referencia_nome_artista)
referencia_nome_musica = ['track_uri']
df_musicas_formatada = transform_fields(df_musica_ordenado, referencia_nome_musica)
referencia_album_name = ['album_uri']
df_musicas_formatada = transform_fields(df_musica_ordenado, referencia_album_name)

In [ ]:
df_musicas_formatada.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1189 entries, 742 to 860
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pos          1189 non-null   int64 
 1   artist_name  1189 non-null   object
 2   track_uri    1189 non-null   int64 
 3   artist_uri   1189 non-null   int64 
 4   track_name   1189 non-null   object
 5   album_uri    1189 non-null   int64 
 6   duration_ms  1189 non-null   int64 
 7   album_name   1189 non-null   object
dtypes: int64(5), object(3)
memory usage: 83.6+ KB


In [ ]:
df_musica_ordenado.head(20)

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
742,0,311,41,394,Down,833,172160,311
380,2,311,115,394,Jackolantern's Weather,833,204160,311
451,4,311,339,394,Too Much To Think,676,234451,MOSAIC
563,49,A Boogie Wit da Hoodie,464,292,Say A',400,214853,The Bigger Artist
535,21,A Boogie Wit da Hoodie,245,292,Drowning (feat. Kodak Black),400,209269,The Bigger Artist
663,77,A Tribe Called Quest,89,17,Electric Relaxation,393,226133,The Anthology
194,39,A.Dd+,594,89,Can't Come Down,931,224549,Can't Come Down
911,59,AFI,552,111,Girl's Not Grey,180,190493,Sing The Sorrow
584,70,AJR,407,647,Let the Games Begin,205,201004,Let the Games Begin
582,68,AJR,444,647,Drama,940,204426,The Click


In [ ]:
# Selecionando variáveis chaves
#df_musica_ordenado = df_musica_ordenado.drop('artist_name', 1)
#df_musica_ordenado = df_musica_ordenado.drop('track_name', 1)
#df_musica_ordenado = df_musica_ordenado.drop('album_name', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [ ]:
df_musica_ordenado

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
742,0,311,41,394,Down,833,172160,311
380,2,311,115,394,Jackolantern's Weather,833,204160,311
451,4,311,339,394,Too Much To Think,676,234451,MOSAIC
563,49,A Boogie Wit da Hoodie,464,292,Say A',400,214853,The Bigger Artist
535,21,A Boogie Wit da Hoodie,245,292,Drowning (feat. Kodak Black),400,209269,The Bigger Artist
...,...,...,...,...,...,...,...,...
608,22,deadmau5,14,206,Telemiscommunications,697,247755,> album title goes here <
397,3,dvsn,65,700,Hallucinations,81,246293,SEPT 5TH
537,23,mansionz,405,269,STFU,679,230878,Mansionz
237,82,salvia palth,1134,91,i was all over her,228,161462,melanchole


# Análise exploratória

In [ ]:
# Analisando alguma correlação entre as variáveis
df_playlist.corr().style.background_gradient(cmap='Blues')

,pid,modified_at,num_tracks,num_albums,num_followers,num_edits,duration_ms,num_artists
pid,1.000000,0.011144,0.037734,0.021650,0.026420,0.033383,0.034757,0.017248
modified_at,0.011144,1.000000,0.184673,0.223503,0.018668,0.229558,0.174858,0.200512
num_tracks,0.037734,0.184673,1.000000,0.904708,0.015867,0.627979,0.988350,0.822444
num_albums,0.021650,0.223503,0.904708,1.000000,-0.001986,0.690599,0.902956,0.956990
num_followers,0.026420,0.018668,0.015867,-0.001986,1.000000,0.003034,0.006578,0.028290
num_edits,0.033383,0.229558,0.627979,0.690599,0.003034,1.000000,0.630271,0.681765
duration_ms,0.034757,0.174858,0.988350,0.902956,0.006578,0.630271,1.000000,0.819133
num_artists,0.017248,0.200512,0.822444,0.956990,0.028290,0.681765,0.819133,1.000000


In [ ]:
X = df_feat
y = df_tracks_format['artist_name']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train, y_train)

In [ ]:
pred = knn.predict(X_test)

In [ ]:
pred